In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

from keras.models import Model, load_model
from keras.utils import multi_gpu_model
from keras.layers import Input, LSTM, Dense, Lambda, GRU, Embedding, GaussianNoise
from keras import backend as K
from keras.callbacks import EarlyStopping, TensorBoard
from keras_exp.multigpu import print_mgpu_modelsummary

import pyodbc
import pandas as pd
import time
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
import json

from pathlib import Path

%matplotlib inline

with open ('../params.json') as f:
    params = json.load(f)
    
table_prefix = params['table_prefix']
diseases = params['diseases']
case_limit = params['case_limit']
control_limit = params['control_limit']
min_enrollment = params['enrollment']
user = params['user']

creds_file = "/home/" + user + "/creds.txt" 
creds = lines = [line.rstrip('\n') for line in open(creds_file)]

connection_string = ("Driver={ODBC Driver 17 for SQL Server};" + 
                     "server=" + creds[0] + ";" +
                     "domain=" + creds[1] + ";" +  
                     "database=" + creds[2] + ";" +
                     "uid=" + creds[3]  + ";" +
                     "pwd=" + creds[4] + ";" +
                     "ssl=require;")

cn = pyodbc.connect(connection_string, autocommit=True)
cursor = cn.cursor()

chunk = True
directory = '../../data/' + str(table_prefix) + '_' + str(case_limit)

/home/anaconda/anaconda3/envs/py27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
NCCL support available


In [27]:
import tensorflow as tf
from keras.callbacks import ModelCheckpoint

print(tf.__version__)
    
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
K.set_session(sess)  # set this TensorFlow session as the default session for Keras


datadir = Path(directory)
files = [(str(f), f.parts[-1]) for f in datadir.glob('*.csv.gz') if f]

1.8.0


In [28]:
# 1a load models
# 1b load in validation sequences 
# 2 hidden layer raw sequences 
# 3 compare distance to other sequences from same person
# 4 % of nearest neighbros that are same class
# 5 fit linear regression from hidden layer to time from index
# 6 scale hidden features by linear regression 
# 7 compare distances to other sequences form same person
# 8 % of nn that are same class

# 1
# load sequence
# predict AE only
# predict GD
# return ae hidden, gd hidden, gd time, y time, disease class
count_seqs = 100
files_df = pd.read_csv(directory + '/files_df.csv', header=None)
files_df.columns = columns=['row', 'path']
print(files_df.shape, files_df.columns)

vocab_dict = pkl.load(open('../../data/diseaes_replacedtest_10000/input_token_index.pkl', 'rb'))
vocab_size = len(vocab_dict)
print(vocab_size)
back_window = 25

# load AE
wname = '../../outputs/test10000ae_only/weights_time2.77-4.86.hdf5'
ae_model = load_model(wname)
ae_model.summary()

layer_name = 'encoded'
ae_hidden_model = Model(inputs=ae_model.input,
                        outputs=ae_model.get_layer(layer_name).output)
ae_hidden_model.summary()

# load time_model
# @TODO update when model finisehd training
wname= '../../outputs/test10000_gd/weights.230-364209.25.hdf5'
gd_model = load_model(wname)
gd_model.summary()

layer_name = 'encoded'
gd_hidden_model = Model(inputs=gd_model.input,
                        outputs=gd_model.get_layer(layer_name).output)
gd_hidden_model.summary()

((2000, 2), Index([u'row', u'path'], dtype='object'))
1436
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input (InputLayer)      (None, None, 1437)   0                                            
__________________________________________________________________________________________________
encoder_input (InputLayer)      (None, None, 1437)   0                                            
__________________________________________________________________________________________________
gaussian_noise_1 (GaussianNoise (None, None, 1437)   0           decoder_input[0][0]              
__________________________________________________________________________________________________
encoded (GRU)                   [(None, 10), (None,  43440       encoder_input[0][0]              
__________________________________________________

In [29]:
query = ("SELECT DISTINCT(MemberNum) FROM bkb12.dbo.test_sequences WHERE Object = '332.0'")
labels = pd.read_sql(query, cn)
pd_list = labels['MemberNum'].tolist()
print(len(pd_list))

10000


In [ ]:
from keras.utils import to_categorical
import time

for x in range(1, 300):
    print(x)
    mem_dict = {} # this may be better as a dataframe - convert after?
    j = 0
    start_time = time.time()
    df = pd.DataFrame(columns=['memberNum', 'ae_hidden', 'gd_hidden', 'gd_time', 'y_time', 'label'])
    for i in range(1731+(10*x), 1741+(10*x)):
        print(i)
        seq = pd.read_csv(files_df.iloc[i].path)
        memberNum = None
        for index, row in seq.iterrows():
            if row['fromIndex'] >= 0 and index < seq.shape[0] - 1:
                j+=1
                label = (0 if row['MemberNum'] in pd_list else 1)

                ae_input_seq = (to_categorical(seq.iloc[index+1-back_window:index+1]['InputCode'].values, 
                                                  num_classes=vocab_size+1))
                gd_input_seq =(to_categorical(seq.iloc[index+1-back_window:index+1]['InputCode'].values, 
                                                  num_classes=vocab_size+1))
                
                ae_input_seq = np.reshape(ae_input_seq, 
                                              (1, ae_input_seq.shape[0], ae_input_seq.shape[1]))
                gd_input_seq = np.reshape(gd_input_seq[:, :-1], 
                                              (1, gd_input_seq.shape[0], gd_input_seq.shape[1]-1))
                try: # any empty sequences
                    ae_hidden = ae_hidden_model.predict([ae_input_seq, ae_input_seq])[0][0]
                    gd_hidden = gd_hidden_model.predict([gd_input_seq, gd_input_seq])[0][0]
                    gd_time = gd_model.predict([gd_input_seq, gd_input_seq])[1][0][0]

                    single_df = pd.DataFrame({'memberNum':[row['MemberNum']], 
                                              'ae_hidden':[ae_hidden],
                                              'gd_hidden':[gd_hidden],
                                              'gd_time':[gd_time],
                                              'y_time':[row['fromIndex']],
                                              'label':[label]})
                    df = df.append(single_df)
                
                except Exception:
                    pass
        
    print(x, len(mem_dict))
    df.to_csv('../../outputs/encoded_dicts/' + str(x) + '.csv')

1
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
(1, 0)
2
1751
1752
1753
1754
1755
1756
1757
1758
1759
1760
(2, 0)
3
1761
1762
1763
1764
1765
1766
1767
1768
1769
1770
(3, 0)
4
1771
1772
1773
1774
1775
1776
1777
1778
1779
1780
(4, 0)
5
1781
1782
1783
1784
1785
1786
1787
1788
1789
1790
(5, 0)
6
1791
1792
1793
1794
1795
1796
1797
1798
1799
1800
(6, 0)
7
1801
1802
1803
1804
1805
1806
1807
1808
1809
1810
(7, 0)
8
1811
1812
1813
1814
1815
1816
1817
1818
1819
1820
(8, 0)
9
1821
1822
1823
1824
1825
1826
1827
1828
1829
1830
(9, 0)
10
1831
1832
1833
1834
1835
1836
1837
1838
1839
1840
(10, 0)
11
1841
1842
1843
1844
1845
1846
1847
1848
1849
1850
(11, 0)
12
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
(12, 0)
13
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
(13, 0)
14
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
(14, 0)
15
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
(15, 0)
16
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
(16, 0)
17
1901
1902
1903
1904
1905
1906
1907
1908

In [ ]:
# load sequences
# for i in range(1601, files.df.shape[0]+1):
# return ae hidden, gd hidden, gd time, y time, label - disease class
from keras.utils import to_categorical
import time

for x in range(1, 400):
    print(x)
    mem_dict = {} # this may be better as a dataframe - convert after?
    j = 0
    start_time = time.time()
    for i in range(1591+(10*x), 1601+(10*x)):
        print(i)
        seq = pd.read_csv(files_df.iloc[i].path)
        # ae_hidden:[], gd_hidden:[], dg_time:[], y_time:[], label:[]
        for index, row in seq.iterrows():
            if row['fromIndex'] >= 0 and index < seq.shape[0] - 1:
                if row['MemberNum'] not in mem_dict.keys():
                    # add to dictionary
                    j+=1
                    curtime = time.time()-start_time
                    avg = (time.time()-start_time)/j
                    print(str(j) + ": " + str(row['MemberNum']) + " " + str(curtime) + " average: " + str(avg))
                    mem_dict[row['MemberNum']] = {'ae_hidden':[], 'gd_hidden':[], 'gd_time':[],
                                                  'y_time':[], 'label':[]}
                    mem_dict[row['MemberNum']]['label'] = (0 if row['MemberNum']
                                                           in pd_list else 1)

                mem_dict[row['MemberNum']]['y_time'].append(row['fromIndex'])

                ae_input_seq = (to_categorical(seq.iloc[index+1-back_window:index+1]['InputCode'].values, 
                                              num_classes=vocab_size+1))
                gd_input_seq =(to_categorical(seq.iloc[index+1-back_window:index+1]['InputCode'].values, 
                                              num_classes=vocab_size+1))

                if ae_input_seq is None or gd_input_seq is None:
                    pass
                else:
                    ae_input_seq = np.reshape(ae_input_seq, 
                                              (1, ae_input_seq.shape[0], ae_input_seq.shape[1]))
                    gd_input_seq = np.reshape(gd_input_seq[:, :-1], 
                                              (1, gd_input_seq.shape[0], gd_input_seq.shape[1]-1))
        #             print(ae_input_seq.shape, gd_input_seq.shape)
                    try:
                        mem_dict[row['MemberNum']]['ae_hidden'].append(ae_hidden_model.predict([ae_input_seq,
                                                                                                ae_input_seq]))
                        mem_dict[row['MemberNum']]['gd_hidden'].append(gd_hidden_model.predict([gd_input_seq,
                                                                                               gd_input_seq]))
                        mem_dict[row['MemberNum']]['gd_time'].append(gd_model.predict([gd_input_seq,
                                                                                      gd_input_seq])[1][0])
                    except:
                        pass

    print(x, len(mem_dict))
    pkl.dump(mem_dict, open('../../outputs/encoded_dicts/mem_dict_' + str(x) +'.pkl', 'wb'))

1
1601
1: 10461916932825 0.016529083252 average: 0.0165300369263
2: 10462466025525 2.00979113579 average: 1.00489652157
3: 10465135344325 8.15978097916 average: 2.71992897987
4: 10467432068525 12.9374120235 average: 3.23435378075
5: 10468955320025 25.6837780476 average: 5.1367562294
6: 10469296678325 29.4639370441 average: 4.91065633297
7: 10470760137825 43.6755161285 average: 6.23935999189
8: 10471384576325 46.8468739986 average: 5.85585951805
9: 10472653639725 49.2234671116 average: 5.46927422947
10: 10473892105425 51.3670890331 average: 5.13670909405
1602
11: 2639553892525 57.6349430084 average: 5.23954036019
12: 2639577380925 70.0931220055 average: 5.84109383821
13: 2640414916325 72.0507509708 average: 5.54236553265
14: 2641056754225 79.4205770493 average: 5.67289865017
15: 2641242965425 85.6117920876 average: 5.70745299657
16: 2641857407525 88.929060936 average: 5.558066383
17: 2644414508825 92.9598119259 average: 5.46822430106
18: 2645427990425 104.451679945 average: 5.8028712802

47: 5148619307225 262.315443993 average: 5.58117967971
48: 5148776415825 281.791263819 average: 5.87065143387
49: 5148831452725 285.396306992 average: 5.82441446733
50: 5149450867025 289.780138969 average: 5.79560281754
1616
51: 4420602906925 298.502700806 average: 5.85299421292
52: 4421257652525 305.124821901 average: 5.8677850549
53: 4421972345625 317.726557016 average: 5.99484081088
54: 4422004997325 337.103839874 average: 6.24266371904
55: 4422694783825 340.436167002 average: 6.18974937959
56: 4424474020525 344.986832857 average: 6.1604792646
57: 4424885417625 350.688997984 average: 6.15243859458
58: 4426202906925 351.437804937 average: 6.05927251536
59: 4426571484725 352.99990201 average: 5.98304921894
60: 4427023420625 358.737880945 average: 5.97896474997
1617
61: 9494203540925 360.470318794 average: 5.90934952361
62: 9495124857625 363.685652018 average: 5.86589762857
63: 9496178290125 367.321694851 average: 5.83050314207
64: 9496380517325 369.343798876 average: 5.77099690586
65:

94: 4145224383025 547.189407825 average: 5.82116398913
95: 4148575237325 584.199738979 average: 6.14947099937
96: 4149280389325 590.118155956 average: 6.14706413448
97: 4149393030425 600.939843893 average: 6.19525613244
98: 4149727307825 603.36123395 average: 6.15674730466
99: 4151297499725 604.512800932 average: 6.10618991804
100: 4152149297325 612.183209896 average: 6.1218321085
(3, 100)
4
1631
1: 19525282796625 0.0574049949646 average: 0.0574059486389
2: 19532144149425 15.1559598446 average: 7.57798051834
3: 19537283367725 18.5211718082 average: 6.17372528712
4: 19595317505625 26.6055719852 average: 6.65139323473
5: 19601564291425 32.2425580025 average: 6.44851198196
6: 19621087820625 38.8048298359 average: 6.46747199694
7: 19622696189025 40.4019730091 average: 5.77171056611
8: 19623684494525 47.8288388252 average: 5.97860500216
9: 19624459968225 51.461261034 average: 5.71791821056
10: 19627060850125 63.1153209209 average: 6.31153249741
1632
11: 2928103539025 64.1850078106 average: 

1645
41: 1703135020825 265.397221088 average: 6.47310307549
42: 1705703942725 274.823132992 average: 6.54340795108
43: 1705853740625 277.776343107 average: 6.45991497816
44: 1706379052825 279.899842024 average: 6.36136009476
45: 1706758630425 280.909058094 average: 6.24242353439
46: 1707043451225 286.814298153 average: 6.2350935003
47: 1707272681425 302.588552952 average: 6.43805434349
48: 1707822476825 305.429245949 average: 6.36310931544
49: 1708206115025 310.49950695 average: 6.33672465597
50: 1710288703825 311.918360949 average: 6.23836724281
1646
51: 5427884788525 313.193367958 average: 6.14104647262
52: 5428347656625 315.217394114 average: 6.06187304167
53: 5428423212625 327.699641943 average: 6.18301217061
54: 5429633262925 329.479585171 average: 6.10147381712
55: 5430220783025 336.058149099 average: 6.11014818278
56: 5430305640625 346.203581095 average: 6.18220687764
57: 5430619272625 351.304650068 average: 6.16323949161
58: 5432615081425 356.896582127 average: 6.15338936345
59

89: 3504872268725 577.603504896 average: 6.48992706685
90: 3505669123025 579.572839975 average: 6.43969823254
1660
91: 13356140330925 580.432776928 average: 6.37838217452
92: 13356771043125 585.181946039 average: 6.36067336798
93: 13356952404525 590.629140854 average: 6.35085098718
94: 13357099049425 592.753098011 average: 6.30588403154
95: 13357183097825 601.358613014 average: 6.33009070346
96: 13358174366125 610.52389884 average: 6.35962397854
97: 13358508426925 612.601183891 average: 6.31547613242
98: 13359310633525 620.995786905 average: 6.33669173474
99: 13359613197025 636.059695005 average: 6.42484544263
100: 13359945920225 654.529335976 average: 6.54529345989
(6, 100)
7
1661
1: 5961767923525 0.0715620517731 average: 0.0715630054474
2: 5964167715125 7.0009431839 average: 3.5004734993
3: 5964899940525 25.8971350193 average: 8.63237937291
4: 5965686768325 26.9280509949 average: 6.73201304674
5: 5966841240125 39.7698960304 average: 7.95398001671
6: 5968117868725 49.715939045 average

...

1740
91: 12272696457625 477.371737003 average: 5.24584327687
92: 12273535431525 489.264239073 average: 5.31808959919
93: 12274382633125 494.501599073 average: 5.31722150567
94: 12274385888025 501.568781137 average: 5.33583810735
95: 12274886946225 503.396279097 average: 5.29890821106
96: 12275081527525 510.484366179 average: 5.31754550089
97: 12275693014425 516.485721111 average: 5.32459510479
98: 12277143797425 523.592904091 average: 5.34278475508
99: 12280850421925 525.591701031 average: 5.30900710279
100: 12281287287525 526.918695211 average: 5.26918696165
(14, 100)
15
1741
1: 10371928982725 0.0328049659729 average: 0.0328071117401
2: 10371981542725 0.0377480983734 average: 0.0188740491867
3: 10374127247825 6.73157310486 average: 2.24385905266
4: 10375606369325 13.1912851334 average: 3.29782176018
5: 10376217619525 23.6382369995 average: 4.72764782906
6: 10376657479725 23.6518311501 average: 3.94197201729
7: 10378184216525 46.3345990181 average: 6.61922901017
8: 10378951647825 52.61

37: 5504587222425 187.784501791 average: 5.07525683738
38: 5504746729125 191.194633961 average: 5.03143789266
39: 5505889585125 203.791171789 average: 5.22541469183
40: 5509640927525 210.745944977 average: 5.2686486721
1755
41: 121222043435361101 238.008341789 average: 5.80508153613
42: 121222053609041201 264.253991842 average: 6.29176188083
43: 121222060309161201 265.131896973 average: 6.16585809131
44: 121222063429051201 266.310689926 average: 6.05251570181
45: 121222071235041201 270.231707811 average: 6.00514911016
46: 121222080732121101 281.929467916 average: 6.12890149718
47: 121222121326141101 287.167127848 average: 6.10993891067
48: 121222140735051101 287.870382786 average: 5.99729966621
49: 121222173427271101 300.872255802 average: 6.14025040549
50: 121222201332341101 306.04240799 average: 6.12084819794
1756
51: 7761626692525 309.306217909 average: 6.06482782083
52: 7761951232225 312.505912781 average: 6.0097291332
53: 7762104364725 326.099738836 average: 6.15282531954
54: 7762

82: 975126152125 471.808604002 average: 5.7537634867
83: 975264745125 482.103197098 average: 5.80847226568
84: 975636978125 483.24763608 average: 5.75294805992
85: 975669779025 485.558821917 average: 5.71245674245
86: 975884410725 485.62698102 average: 5.6468253607
87: 976252775525 494.283737898 average: 5.68142228839
88: 976958289125 496.088083029 average: 5.63736459071
89: 977295922225 497.383865118 average: 5.58858276485
90: 977412877925 500.891811132 average: 5.56546457873
1770
91: 9670022163025 520.835690022 average: 5.72346916566
92: 9670569440325 524.975809097 average: 5.70625881527
93: 9672344445125 525.427566051 average: 5.649758785
94: 9672365600325 530.01781702 average: 5.63848742526
95: 9672965600325 535.535320044 average: 5.63721390523
96: 9675238560225 537.542478085 average: 5.59940084318
97: 9675245497925 540.770060062 average: 5.57494908264
98: 9675654825425 547.541040897 average: 5.58715349071
99: 9675742163025 548.893373013 average: 5.54437751481
100: 9676135807825 55

29: 814629041325 130.532876968 average: 4.50113382833
30: 815078072625 140.020323038 average: 4.66734413306
1784
31: 33585175574625 142.602257013 average: 4.60007286841
32: 34126820770225 151.953904867 average: 4.74855965376
33: 35389276336325 158.47538805 average: 4.80228460196
34: 36382645346125 162.792541981 average: 4.78801596866
35: 36729859662125 164.903683901 average: 4.71153385299
36: 37373045991725 167.50522089 average: 4.65292282899
37: 37394395472325 171.154227018 average: 4.62578997096
38: 39514238743725 171.462914944 average: 4.5121819973
39: 44710748837425 172.172704935 average: 4.41468476638
40: 45385068134325 172.914881945 average: 4.32287207246
1785
41: 231512130735331301 187.508112907 average: 4.57336870635
42: 231512130803213201 196.433771849 average: 4.67699459621
43: 231512130803343401 200.63438797 average: 4.66591602148
44: 231512130805282801 203.081734896 average: 4.61549402367
45: 231512130812033301 204.341045856 average: 4.54091215663
46: 231512130814013601 208

76: 6753295913625 525.260571003 average: 6.91132331522
77: 6754371573425 537.600600958 average: 6.9818260391
78: 6754430864025 541.143336058 average: 6.93773508989
79: 6756610827825 543.524769068 average: 6.88006038002
80: 6756870335025 544.088396072 average: 6.80110496283
1799
81: 12361562078425 552.435493946 average: 6.82019129506
82: 12366121921625 554.701700926 average: 6.76465491551
83: 12366988769125 560.009819984 average: 6.74710631371
84: 12367819803225 560.694371939 average: 6.67493301062
85: 12368675174425 563.137991905 average: 6.62515285997
86: 12369269198425 568.851635933 average: 6.61455391729
87: 12369722092325 573.821393013 average: 6.59564820651
88: 12370850931225 575.858677864 average: 6.54384863648
89: 12372662902425 591.927598953 average: 6.65087192246
90: 12375383201125 597.203713894 average: 6.63559683429
1800
91: 18698614374725 623.745804071 average: 6.85434953721
92: 18698654473525 630.216387033 average: 6.85017813029
93: 18700227753425 631.67850709 average: 6.7